In [1]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import SphericalVoronoi
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib import colors

class Point():
    """3次元直交座標の点のクラス"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x, self.y, self.z = float(x), float(y), float(z)
    
    def distance(self, target=Point()):
        return math.sqrt((self.x - target.x) ** 2 + (self.y - target.y) ** 2 + (self.z - target.z) ** 2)

    def vector_to(self, target=Point()):
        return Point(target.x - self.x, target.y - self.y, target.z - self.z)
    
    def to_list(self):
        return [self.x, self.y, self.z]

class Sphere():
    """球のクラス"""
    def __init__(self, center=Point(0, 0, 0), radius=1.0):
        self.center, self.radius = center, float(radius)
        self.points = []

    def plot(self, elev=30, azim=30):
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        u,v=np.mgrid[0:2*np.pi:50j, 0:np.pi:25j]
        x=np.cos(u)*np.sin(v)
        y=np.sin(u)*np.sin(v)
        z=np.cos(v)
        ax.plot_wireframe(x, y, z, color='lightskyblue', linewidth=0.5)

        x = np.array([])
        y = np.array([])
        z = np.array([])
        for point in self.points:
            x = np.append(x, point.x)
            y = np.append(y, point.y)
            z = np.append(z, point.z)
        ax.plot(x, y, z, marker='o', linestyle='None',)
        ax.view_init(elev=elev, azim=azim)
        plt.show()
        
    def add_random_point(self, number=1):
        """numberの数だけ球面上にランダムな点を加える"""
        points = []
        for _ in range(number):
            theta = random.random() * math.pi
            phi = random.random() * math.pi * 2
            z = random.random() * 2 - 1
            self.points.append(Point(self.radius * math.sqrt(1 - (z ** 2)) * math.cos(phi),
                                     self.radius * math.sqrt(1 - (z ** 2)) * math.sin(phi),
                                     self.radius * z))
            
    def bind_point_on_surface(self, point: Point) -> Point:
        distance = point.distance(self.center)
        factor = self.radius / distance
        return Point(point.x * factor, point.y * factor, point.z * factor)
    
    def bind_all_points(self):
        for i, point in enumerate(self.points):
            self.points[i] = self.bind_point_on_surface(point)
    
    def move(self):
        """最も近い点に定力で反発する"""
        new_points = []
        for point in self.points:
            nearest = Point(999,999,999)
            for target in self.points:
                if point == target:
                    continue
                if point.distance(target) < point.distance(nearest):
                    nearest = target
            v = point.vector_to(nearest)
            size = v.distance()
            vn = Point(v.x / size, v.y / size, v.z / size)
            factor = 1
            new_points.append(Point(point.x - vn.x * factor, point.y - vn.y * factor, point.z - vn.z * factor))
        self.points = new_points
    
    def move2(self):
        """全ての点に対し距離の二乗に反比例して反発する"""
        new_points = []
        for point in self.points:
            vector = Point()
            constant = 1
            for target in self.points:
                if point == target:
                    continue
                v = point.vector_to(target)
                vector.x += -1 * v.x * constant / (v.distance() ** 2)
                vector.y += -1 * v.y * constant / (v.distance() ** 2)
                vector.z += -1 * v.z * constant / (v.distance() ** 2)
            new_points.append(Point(point.x + vector.x, point.y + vector.y, point.z + vector.z))
        self.points = new_points
        
    def move3(self, power=2):
        """全ての点に対し距離に反比例して反発する"""
        new_points = []
        for point in self.points:
            vector = Point()
            constant = 1
            for target in self.points:
                if point == target:
                    continue
                v = point.vector_to(target)
                vector.x += -1 * v.x * constant / (v.distance() ** power)
                vector.y += -1 * v.y * constant / (v.distance() ** power)
                vector.z += -1 * v.z * constant / (v.distance() ** power)
            new_points.append(Point(point.x + vector.x, point.y + vector.y, point.z + vector.z))
        self.points = new_points
        
    def voronoi(self, plot=False):
        """球面ボロノイによる評価"""
        areas = []
        points = np.array([p.to_list() for p in self.points])
        center = np.array(self.center.to_list())
        sv = SphericalVoronoi(points, self.radius, center)
        # 面積を計算
        for region in sv.regions:
            triangles = []
            number_of_triangles = len(region) - 2
            for n in range(number_of_triangles):
                triangles.append([region[0], region[n+1], region[n+2]])
            angle = 0
            #print(triangles)
            for triangle in triangles:
                O = np.array([self.center.x, self.center.y, self.center.z])
                A = sv.vertices[triangle[0]] - O
                B = sv.vertices[triangle[1]] - O
                C = sv.vertices[triangle[2]] - O
                cAB = np.cross(A, B)
                cAB = cAB / np.linalg.norm(cAB)
                cBC = np.cross(B, C)
                cBC = cBC / np.linalg.norm(cBC)
                cCA = np.cross(C, A)
                cCA = cCA / np.linalg.norm(cCA)
                a = 0
                a += math.acos(np.dot(cAB, -cBC))
                a += math.acos(np.dot(cBC, -cCA))
                a += math.acos(np.dot(cCA, -cAB))
                a -= math.pi
                angle += a
                #print(a)
            areas.append(angle)
        #print(areas)
            
            
        if plot:
            # sort vertices (optional, helpful for plotting)
            sv.sort_vertices_of_regions()
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(111, projection='3d')
            # plot the unit sphere for reference (optional)
            u = np.linspace(0, 2 * np.pi, 100)
            v = np.linspace(0, np.pi, 100)
            x = np.outer(np.cos(u), np.sin(v))
            y = np.outer(np.sin(u), np.sin(v))
            z = np.outer(np.ones(np.size(u)), np.cos(v))
            ax.plot_surface(x, y, z, color='y', alpha=0.1)
            # plot generator points
            ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='b')
            # plot Voronoi vertices
            ax.scatter(sv.vertices[:, 0], sv.vertices[:, 1], sv.vertices[:, 2], c='g')
            # indicate Voronoi regions (as Euclidean polygons)
            for region in sv.regions:
                random_color = colors.rgb2hex(np.random.rand(3))
                polygon = Poly3DCollection([sv.vertices[region]], alpha=1.0)
                polygon.set_color(random_color)
                ax.add_collection3d(polygon)
            plt.show()
        return areas
    
            
sphere = Sphere(center=Point(0,0,0), radius=10.0)
sphere.add_random_point(8)
sphere.plot()
#sphere.plot(30, 120)
#for p in sphere.points:
#    print(p.distance(sphere.center))
for i in range(100):
    sphere.move3(power=2)
    #sphere.plot()
    sphere.bind_all_points()
#    if i % 10 == 0:
#        sphere.plot()
sphere.plot()
areas = sphere.voronoi(plot=True)
print(f"平均{np.mean(areas)}")
print(f"標準偏差{np.std(areas)}")

NameError: name 'Point' is not defined